Welcome!  If you are new to Google Colab/Jupyter notebooks, you might take a look at [this notebook](https://colab.research.google.com/notebooks/basic_features_overview.ipynb) first.

**I recommend you run the first code cell of this notebook immediately, to start provisioning drake on the cloud machine, then you can leave this window open as you [read the textbook](http://underactuated.csail.mit.edu/sysid.html).**

# Notebook Setup

The following cell will:
- on Colab (only), install Drake to `/opt/drake`, install Drake's prerequisites via `apt`, and add pydrake to `sys.path`.  This will take approximately two minutes on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  If you navigate between notebooks using Colab's "File->Open" menu, then you can avoid provisioning a separate machine for each notebook.
- import packages used throughout the notebook.

You will need to rerun this cell if you restart the kernel, but it should be fast (even on Colab) because the machine will already have drake installed.

In [ ]:
import importlib
import sys
from urllib.request import urlretrieve

# Install drake (and underactuated).
if 'google.colab' in sys.modules and importlib.util.find_spec('underactuated') is None:
    urlretrieve(f"http://underactuated.csail.mit.edu/scripts/setup/setup_underactuated_colab.py",
                "setup_underactuated_colab.py")
    from setup_underactuated_colab import setup_underactuated
    setup_underactuated(underactuated_sha='ad5af65f9010d47ddfcd71b1ab0241570f766380', drake_version='0.27.0', drake_build='release')

server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

import numpy as np
from ipywidgets import FloatSlider, ToggleButton
from IPython.display import display, SVG
import pydot

from underactuated.jupyter import running_as_notebook

np.set_printoptions(suppress=True)

# LQR for the Acrobot with joint offsets

Adding only a very small offset (my default is 0.01 radians, or approximately half a degree for each joint) can lead to quite large steady-state errors.

TODO: Make a plot of offset magnitude to steady-state error.

In [ ]:
import pydrake.all
from pydrake.all import (
    DiagramBuilder, LinearQuadraticRegulator, Saturation, SceneGraph, Simulator, 
    WrapToSystem, AddMultibodyPlantSceneGraph, Parser, Adder, ConstantVectorSource,
)
from pydrake.examples.acrobot import (AcrobotInput, AcrobotGeometry, 
                                      AcrobotPlant, AcrobotState)


def acrobot_balancing_example(offsets=[0.01,0.01]):
    builder = DiagramBuilder()

    def UprightState():
        state = AcrobotState()
        state.set_theta1(np.pi)
        state.set_theta2(0.)
        state.set_theta1dot(0.)
        state.set_theta2dot(0.)
        return state

    def BalancingLQR():
        # Design an LQR controller for stabilizing the Acrobot around the upright.
        # Returns a (static) AffineSystem that implements the controller (in
        # the original AcrobotState coordinates).

        acrobot = AcrobotPlant()
        context = acrobot.CreateDefaultContext()

        input = AcrobotInput()
        input.set_tau(0.)
        acrobot.get_input_port(0).FixValue(context, input)

        context.get_mutable_continuous_state_vector()\
            .SetFromVector(UprightState().CopyToVector())

        Q = np.diag((10., 10., 1., 1.))
        R = [1]

        return LinearQuadraticRegulator(acrobot, context, Q, R)


    acrobot = builder.AddSystem(AcrobotPlant())
    acrobot.set_name('acrobot')

    saturation = builder.AddSystem(Saturation(min_value=[-10], max_value=[10]))
    saturation.set_name('saturation')
    builder.Connect(saturation.get_output_port(0), acrobot.get_input_port(0))
    wrapto = builder.AddSystem(WrapToSystem(4))
    wrapto.set_name('wrap_angles')
    wrapto.set_interval(0, 0, 2. * np.pi)
    wrapto.set_interval(1, -np.pi, np.pi)
    builder.Connect(acrobot.get_output_port(0), wrapto.get_input_port(0))
    controller = builder.AddSystem(BalancingLQR())
    controller.set_name("lqr controller")

    # I'll add the offsets in here:
    adder = builder.AddSystem(Adder(2, 4))
    adder.set_name("adder")
    builder.Connect(wrapto.get_output_port(), adder.get_input_port(0))
    builder.Connect(adder.get_output_port(), controller.get_input_port())
    offset = builder.AddSystem(ConstantVectorSource(np.array(offsets + [0, 0])))
    offset.set_name("state calibration offsets")
    builder.Connect(offset.get_output_port(), adder.get_input_port(1))

    builder.Connect(controller.get_output_port(), saturation.get_input_port())
    # Setup visualization
    scene_graph = builder.AddSystem(SceneGraph())
    AcrobotGeometry.AddToBuilder(builder, acrobot.get_output_port(0), scene_graph)
    visualizer = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(
        builder, 
        scene_graph=scene_graph, 
        zmq_url=zmq_url)
    visualizer.vis.delete()
    visualizer.set_planar_viewpoint(xmin=-3.0, xmax=3.0, ymin=-3.0, ymax=4.0)

    diagram = builder.Build()

    display(SVG(pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].create_svg()))

    # Set up a simulator to run this diagram
    simulator = Simulator(diagram)
    simulator.set_target_realtime_rate(1.0)
    context = simulator.get_mutable_context()

    # Simulate
    duration = 4.0 if running_as_notebook else 0.1 # sets a shorter duration during testing
    for i in range(5):
        context.SetTime(0.)
        context.SetContinuousState(UprightState().CopyToVector() +
                                0.05 * np.random.randn(4,))
        simulator.Initialize()
        simulator.AdvanceTo(duration)

acrobot_balancing_example()

# Extracting the lumped parameters from the Cart-Pole

Recall that the dynamics of our cart-pole system were

\begin{gather}
(m_c + m_p)\ddot{x} + m_p l \ddot\theta \cos\theta - m_p l \dot\theta^2 \sin\theta = f_x \\
m_p l \ddot{x} \cos\theta + m_p l^2 \ddot\theta + m_p g l \sin\theta = 0
\end{gather}

Note: This example requires https://github.com/RobotLocomotion/drake/pull/14863 , which is not available in the drake binaries quite yet.

In [ ]:
import matplotlib.pyplot as plt
from pandas import DataFrame
from pydrake.all import (
    MultibodyPlant, Parser, MakeVectorVariable, MultibodyForces_, SpatialInertia_, UnitInertia_,
    Expression, Variable, Polynomial, Variables, DecomposeLumpedParameters, Evaluate,
    PiecewisePolynomial, TrajectorySource,
    DiagramBuilder, Simulator, LogOutput
) 
from underactuated import FindResource

def cartpole_generate_data():
    fig, ax = plt.subplots(2, 2)
    ax[0,0].set_xlabel('t')
    ax[0,0].set_ylabel('x')
    ax[1,0].set_xlabel('t')
    ax[1,0].set_ylabel('theta')
    ax[0,1].set_xlabel('t')
    ax[0,1].set_ylabel('xdot')
    ax[1,1].set_xlabel('t')
    ax[1,1].set_ylabel('thetadot')

    data = []

    for i in range(4):
        ts = np.arange(0, 4, 0.1)
        utraj = PiecewisePolynomial.CubicShapePreserving(ts, 10*np.sin((i+1)*ts).reshape((1,-1)))

        builder = DiagramBuilder()

        plant = builder.AddSystem(MultibodyPlant(0.0))
        file_name = FindResource("models/cartpole.urdf")
        Parser(plant).AddModelFromFile(file_name)
        plant.Finalize()

        usys = builder.AddSystem(TrajectorySource(utraj)) 
        builder.Connect(usys.get_output_port(), plant.get_actuation_input_port())

        logger = LogOutput(plant.get_state_output_port(), builder)
        diagram = builder.Build()

        simulator = Simulator(diagram)
        simulator.AdvanceTo(ts[-1])

        data.append(DataFrame({
            't': logger.sample_times(), 
            'u': utraj.vector_values(logger.sample_times())[0,:],
            'x': logger.data()[0,:],
            'theta': logger.data()[1,:],
            'xdot': logger.data()[2,:],
            'thetadot': logger.data()[3,:],
            }))

        ax[0,0].plot(data[i].t, data[i].x.T)
        ax[1,0].plot(data[i].t, data[i].theta.T)
        ax[0,1].plot(data[i].t, data[i].xdot.T)
        ax[1,1].plot(data[i].t, data[i].thetadot.T)

    return data
        

data = cartpole_generate_data()


In [ ]:
def cartpole_estimate_lumped_parameters(data):
    plant = MultibodyPlant(0.0)
    file_name = FindResource("models/cartpole.urdf")
    Parser(plant).AddModelFromFile(file_name)
    plant.Finalize()

    sym_plant = plant.ToSymbolic()
    sym_context = sym_plant.CreateDefaultContext()

    # State variables
    q = MakeVectorVariable(2, "q")
    v = MakeVectorVariable(2, "v")
    vd = MakeVectorVariable(2, "vd")
    tau = MakeVectorVariable(1, "u")

    # Parameters
    mc = Variable("mc")
    mp = Variable("mp")
    l = Variable("l")

    sym_plant.get_actuation_input_port().FixValue(sym_context, tau)
    sym_plant.SetPositions(sym_context, q)
    sym_plant.SetVelocities(sym_context, v)

    cart = sym_plant.GetBodyByName("cart")
    cart.SetMass(sym_context, mc)
    pole = sym_plant.GetBodyByName("pole")
    inertia = SpatialInertia_[Expression](mp, [0, 0, -l], UnitInertia_[Expression](l*l,l*l,0))
    pole.SetSpatialInertiaInBodyFrame(sym_context, inertia)

    # TODO: Set the joint damping pending resolution of drake #14405
    # x = sym_plant.GetJointByName("x")
    # theta = sym_plant.GetJointByName("theta")

    forces = MultibodyForces_[Expression](sym_plant)
    sym_plant.CalcForceElementsContribution(sym_context, forces)
    error = sym_plant.CalcInverseDynamics(sym_context, vd, forces) - sym_plant.MakeActuationMatrix() @ tau

    print('Symbolic dynamics: ')
    print(error[0].Expand())
    print(error[1].Expand())
    print('')

    W, alpha, w0 = DecomposeLumpedParameters(error, [mc, mp, l])

    # Form the data matrix 
    # TODO: Update this to the vectorized version pending drake #8495
#    env = {
#        q[0]: np.concatenate([d.x[:-1] for d in data]),
#        q[1]: np.concatenate([d.theta[:-1] for d in data]),
#        v[0]: np.concatenate([d.xdot[:-1] for d in data]),
#        v[1]: np.concatenate([d.thetadot[:-1] for d in data]),
#        vd[0]: np.concatenate([(d.xdot[1:]-d.xdot[:-1])/h for d in data]),
#        vd[1]: np.concatenate([(d.thetadot[1:]-d.thetadot[:-1])/h for d in data]),
#        tau[0]:np.concatenate([d.u[:-1] for d in data])
#    } 
#    Wdata = Evaluate(W, env)
    M = data[0].t.shape[0] - 1
    MM = 2*M*len(data)
    N = alpha.size
    Wdata = np.zeros((MM, N))
    w0data = np.zeros((MM, 1))
    offset = 0
    for d in data:
        for i in range(M):
            h = d.t[i+1]-d.t[i]
            env = {
                q[0]: d.x[i],
                q[1]: d.theta[i],
                v[0]: d.xdot[i],
                v[1]: d.thetadot[i],
                vd[0]: (d.xdot[i+1]-d.xdot[i])/h,
                vd[1]: (d.thetadot[i+1]-d.thetadot[i])/h,
                tau[0]: d.u[i],
            }
            
            Wdata[offset:offset+2,:] = Evaluate(W, env)
            w0data[offset:offset+2] = Evaluate(w0, env)
            offset += 2

    alpha_fit = np.linalg.lstsq(Wdata, -w0data, rcond=None)[0]
    alpha_true = Evaluate(alpha, {mc: 10, mp: 1, l: .5})

    print('Estimated Lumped Parameters:')
    for i in range(len(alpha)):
        print(f"{alpha[i]}.  URDF: {alpha_true[i,0]},  Estimated: {alpha_fit[i,0]}")

cartpole_estimate_lumped_parameters(data)